In [2]:
import json
import re

data = []
with open('dump_small.jsonln', 'r') as file:
    for line in file:
        data.append(json.loads(line))
        
print(f'Numero de documentos: {len(data)}')

Numero de documentos: 11225


In [3]:
print(data[0].keys())

dict_keys(['title', 'body'])


In [4]:
print(data[0]['title'])

Alexandre (nome)


In [5]:
print(data[0]['body'])

{{Multitag|mnot|rev|data=outubro de 2019}}
{{Info/Nome
|imagem= Napoli BW 2013-05-16 16-24-01.jpg
|legenda= [[Alexandre, o Grande]], uma das pessoas notórias de nome '''Alexandre'''.
|gênero= Masculino
|significado= Protetor do Homem
|origem= Do [[Língua grega|grego]] Αλέξανδρος
|relativos= Alex; Alexander; Alessandro; Sandro
}}
'''Alexandre''' é um [[prenome]] popular da [[língua portuguesa]]. É [[cognato]] ao nome [[Alexander]], da [[língua inglesa]]. Em [[países lusófonos]], pessoas chamadas Alexandre são normalmente [[Alcunha|apelidadas]] de [[Alex]].

== Origem ==
O nome deriva do [[latim]] ''Alexander'', [[romanização]] do nome [[Língua grega|grego]] Αλέξανδρος (''Aléksandros''). [[Etimologia|Etimologicamente]], o nome seria um composto do [[verbo]] ''ἀλέξειν'' (''aléksein''), "defender", e o [[substantivo]] ''ἀνδρός'' (''andrós''), [[genitivo]] de ''ἀνήρ'' (''anēr''), "homem". Assim, pode ser traduzido como "protetor da Humanidade".{{carece de fontes|data=abril de 2017}}

O term

---

Como vocês podem ver, o texto está cheio de caracteres de estruturação de documento.

---

**Atividade:** Consulte a documentação da Wikipedia para descobrir o que são as marcações `[[`, `]]`, `{{` e ```}}```

**R:** O que está entre [[ ]] é um hypertexto. o que está entre {{}} é um supertexto (ex: aaaa², o 2 é um supertexto)

---

Vamos explorar o corpus para minerar itens de interesse.

**Exercicio:** Faça uma função que retorna uma lista com todos os links html de um texto deste corpus.

In [6]:
def find_links(text):
    return re.findall(r'\[\[.+?\]\]', text)

In [7]:
print(find_links(data[0]['body']))

['[[Alexandre, o Grande]]', '[[Língua grega|grego]]', '[[prenome]]', '[[língua portuguesa]]', '[[cognato]]', '[[Alexander]]', '[[língua inglesa]]', '[[países lusófonos]]', '[[Alcunha|apelidadas]]', '[[Alex]]', '[[latim]]', '[[romanização]]', '[[Língua grega|grego]]', '[[Etimologia|Etimologicamente]]', '[[verbo]]', '[[substantivo]]', '[[genitivo]]', '[[tatpurusha]]', '[[cognato]]', '[[sânscrito]]', '[[Ramayana]]', '[[Proto-Indo-Europeu|PIE]]', '[[terpsimbrotos]]', '[[Língua grega micénica|grego micênico]]', '[[Linear B]]', '[[epíteto]]', '[[Divindade|deusa]]', '[[Mitologia grega|grega]]', '[[Hera]]', '[[Ilíada]]', '[[Páris]]', '[[Alexandre, o Grande|Alexandre III]]', '[[Macedónia Antiga|Macedônia]]', '[[Rússia]]', '[[czar]]', '[[Alexandre I da Rússia]]', '[[Língua albanesa|Albanês]]', '[[Skanderbeg]]', '[[Língua amárica|Amárico]]', '[[Língua árabe|Árabe]]', '[[Língua bielorrussa|Bielorrusso]]', '[[Língua catalã|Catalão]]', '[[Língua inglesa|Inglês]]', '[[Língua gaélica escocesa|Gaélico 

**Exercicio:** Faça uma função que recebe um texto do corpus e substitui todas as ocorrências de links da Wikipedia (textos entre os tags `[[` e `]]`) por texto simples, e retorna o documento limpo. Por exemplo, quando encontrar algo como `[[Etimologia|Etimologicamente]]` substituir por `Etimologicamente`. Você pode supor que os links não são *aninháveis*, como por exemplo `[[blablabla[[etc]]blebleble]]`.

In [8]:
def replace_links(text):
    
    clean = text
    
    links = find_links(text)
    words = []
    for i in links:
        a = re.search(r'(?<=\[\[).+?(?=\]\])', i)
        if '|' in a.group(0):
            b = a.group(0).find('|') + 1
        else:
            b = 0
        words.append(a.group(0)[b:])

    for i in range(len(links)):
        clean = clean.replace(links[i], words[i])
    
    return clean

In [9]:
replace_links(data[0]['body'])

'{{Multitag|mnot|rev|data=outubro de 2019}}\n{{Info/Nome\n|imagem= Napoli BW 2013-05-16 16-24-01.jpg\n|legenda= Alexandre, o Grande, uma das pessoas notórias de nome \'\'\'Alexandre\'\'\'.\n|gênero= Masculino\n|significado= Protetor do Homem\n|origem= Do grego Αλέξανδρος\n|relativos= Alex; Alexander; Alessandro; Sandro\n}}\n\'\'\'Alexandre\'\'\' é um prenome popular da língua portuguesa. É cognato ao nome Alexander, da língua inglesa. Em países lusófonos, pessoas chamadas Alexandre são normalmente apelidadas de Alex.\n\n== Origem ==\nO nome deriva do latim \'\'Alexander\'\', romanização do nome grego Αλέξανδρος (\'\'Aléksandros\'\'). Etimologicamente, o nome seria um composto do verbo \'\'ἀλέξειν\'\' (\'\'aléksein\'\'), "defender", e o substantivo \'\'ἀνδρός\'\' (\'\'andrós\'\'), genitivo de \'\'ἀνήρ\'\' (\'\'anēr\'\'), "homem". Assim, pode ser traduzido como "protetor da Humanidade".{{carece de fontes|data=abril de 2017}}\n\nO termo seria ou um tipo raro de composto "tatpurusha invert

**Exercicio**: Faça uma função que recebe um texto do corpus e remove todas as ocorrências de referências (textos entre os tags `<ref>` e `</ref>`).

In [10]:
def remove_ref(text):
    return re.sub(r'<ref>.+</ref>', '', text)

In [11]:
remove_ref(replace_links(data[0]['body']))

'{{Multitag|mnot|rev|data=outubro de 2019}}\n{{Info/Nome\n|imagem= Napoli BW 2013-05-16 16-24-01.jpg\n|legenda= Alexandre, o Grande, uma das pessoas notórias de nome \'\'\'Alexandre\'\'\'.\n|gênero= Masculino\n|significado= Protetor do Homem\n|origem= Do grego Αλέξανδρος\n|relativos= Alex; Alexander; Alessandro; Sandro\n}}\n\'\'\'Alexandre\'\'\' é um prenome popular da língua portuguesa. É cognato ao nome Alexander, da língua inglesa. Em países lusófonos, pessoas chamadas Alexandre são normalmente apelidadas de Alex.\n\n== Origem ==\nO nome deriva do latim \'\'Alexander\'\', romanização do nome grego Αλέξανδρος (\'\'Aléksandros\'\'). Etimologicamente, o nome seria um composto do verbo \'\'ἀλέξειν\'\' (\'\'aléksein\'\'), "defender", e o substantivo \'\'ἀνδρός\'\' (\'\'andrós\'\'), genitivo de \'\'ἀνήρ\'\' (\'\'anēr\'\'), "homem". Assim, pode ser traduzido como "protetor da Humanidade".{{carece de fontes|data=abril de 2017}}\n\nO termo seria ou um tipo raro de composto "tatpurusha invert

**Exercicio:** Faça uma função que recebe um texto do corpus e remove todos os templates (textos entre os tags `{{` e `}}`). Este exercício é desafiante: *os templates podem ser aninhados*! E agora, como proceder?

In [12]:
def remove_templates(text):
    a = text
    while '{' in a:
        to_replace = re.findall(r'\{\{[^\{\{\}\}]*\}\}', a)
        for i in to_replace:
            a = a.replace(i, '')
    
    return a

In [13]:
remove_templates(remove_ref(replace_links(data[0]['body'])))

'\n\n\'\'\'Alexandre\'\'\' é um prenome popular da língua portuguesa. É cognato ao nome Alexander, da língua inglesa. Em países lusófonos, pessoas chamadas Alexandre são normalmente apelidadas de Alex.\n\n== Origem ==\nO nome deriva do latim \'\'Alexander\'\', romanização do nome grego Αλέξανδρος (\'\'Aléksandros\'\'). Etimologicamente, o nome seria um composto do verbo \'\'ἀλέξειν\'\' (\'\'aléksein\'\'), "defender", e o substantivo \'\'ἀνδρός\'\' (\'\'andrós\'\'), genitivo de \'\'ἀνήρ\'\' (\'\'anēr\'\'), "homem". Assim, pode ser traduzido como "protetor da Humanidade".\n\nO termo seria ou um tipo raro de composto "tatpurusha invertido", com o modificante na segunda posição (o tatpurusha cognato em sânscrito sendo *nararakṣa, cf. Ramayana 6.33.45; o equivalente exato em sânscrito seria *rakṣinara, do PIE \'\'hleks(i)-hnros\'\') ou um composto gasto do tipo terpsimbrotos, cujo significado original seria "ele protege os homens".\n\nO primeiro registro conhecido do nome foi feito no grego

**Exercicio:** Usando os codigos desenvolvidos acima, faça uma função que recebe um texto do corpus e retorna a sua versão limpa.

In [14]:
def clear_text(text):
    return remove_templates(remove_ref(replace_links(text)))

In [15]:
clear_text(data[0]['body'])

'\n\n\'\'\'Alexandre\'\'\' é um prenome popular da língua portuguesa. É cognato ao nome Alexander, da língua inglesa. Em países lusófonos, pessoas chamadas Alexandre são normalmente apelidadas de Alex.\n\n== Origem ==\nO nome deriva do latim \'\'Alexander\'\', romanização do nome grego Αλέξανδρος (\'\'Aléksandros\'\'). Etimologicamente, o nome seria um composto do verbo \'\'ἀλέξειν\'\' (\'\'aléksein\'\'), "defender", e o substantivo \'\'ἀνδρός\'\' (\'\'andrós\'\'), genitivo de \'\'ἀνήρ\'\' (\'\'anēr\'\'), "homem". Assim, pode ser traduzido como "protetor da Humanidade".\n\nO termo seria ou um tipo raro de composto "tatpurusha invertido", com o modificante na segunda posição (o tatpurusha cognato em sânscrito sendo *nararakṣa, cf. Ramayana 6.33.45; o equivalente exato em sânscrito seria *rakṣinara, do PIE \'\'hleks(i)-hnros\'\') ou um composto gasto do tipo terpsimbrotos, cujo significado original seria "ele protege os homens".\n\nO primeiro registro conhecido do nome foi feito no grego

**Exercicio:** Limpe todos os documentos e explore os resultados para ver o que mais dá para limpar. Nosso objetivo é ter uma coletânea de textos limpos para poder criar um vocabulário da língua portuguesa!

In [18]:
print(data[1]['body'])

{{Mais notas|data=novembro de 2019}}
{{Ver desambig}}
{{Info/Município de Portugal
|município = Aveiro
|imagem_brasão = AVR.png
|imagem_bandeira = Pt-avr1.png
|imagem_montagem ={{Photomontage
| photo1a                 = Ilha Dos Puxadoiros (47261194681) (cropped).jpg
| photo2a                 = Aveiro Foto (cropped).jpg
| photo2b                 = Passeio de moliceiro (cropped).jpg
| photo3a                 = Antiga Capitania do Porto de Aveiro - Portugal (5679861484) (cropped).jpg
| photo3b                 = Conjunto de Armazéns do Sal do Canal de São Roque ou Palheiros de sal do Canal de São Roque - envolvente 3 (cropped).jpg
| photo4a                 = 
| spacing                 = 2
| position                = centre
| color_border            = white
| color                   = white
| size                    = 280
| foot_montage            =
}}
|imagem_localização = LocalAveiro.svg
|gentílico = [[wikt:Aveirense|Aveirense]]
|região = [[Região do Centro (Portugal)|Centro]]
|subregião

In [19]:
print(clear_text(data[1]['body']))

KeyboardInterrupt: 

# Criando um tokenizador

Agora que temos um conjunto de documentos limpos, vamos analisar esses documentos para:

- Separar os documentos em palavras

- Separar os documentos em frases

**Exercicio:** Faça uma função que separa os documentos em palavras usando simplesmente os espaços em branco como indicador de separação.

**Exercicio:** Agora analise os resultados obtidos: essa separação em palavras está perfeita? Certamente não:

- Um problema óbvio é o seguinte: palavras ao final de uma sentença podem carregar consigo a pontuação da frase. 

- Mais ainda, algumas palavras no meio das sentenças podem ter vírgula e ponto-e-vírgula juntos.

- Sequências de emojis podem aparecer, e cada emoji deve ser considerado como uma palavra separada

Faça uma função que recebe um documento e faz uma separação melhor.